![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Hardcore DL by Spark NLP

In [ ]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.4.0

## Explain Documents with Deep Learning

In [1]:
import sys
import time

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import *

### Let's create a Spark Session for our app

Let's take a look at what's behind `sparknlp.start()` function:

In [2]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.0
Apache Spark version:  2.4.4


In [3]:
pipeline = PretrainedPipeline('explain_document_dl', lang='en')

explain_document_dl download started this may take some time.
Approx size to download 168.4 MB
[OK!]


We simply send the text we want to transform and the pipeline does the work.

In [4]:
text = 'He would love to visit many beautful cities wth you. He lives in an amazing country like Germany or Pakistan.'
result = pipeline.annotate(text)

We can see the output of each annotator below. This one is doing so many things at once!

In [5]:
list(result.keys())

['entities',
 'stem',
 'checked',
 'lemma',
 'document',
 'pos',
 'token',
 'ner',
 'embeddings',
 'sentence']

In [6]:
result['sentence']

['He would love to visit many beautful cities wth you.',
 'He lives in an amazing country.']

In [7]:
result['lemma']

['He',
 'would',
 'love',
 'to',
 'visit',
 'many',
 'beautiful',
 'city',
 'wth',
 'you',
 '.',
 'He',
 'life',
 'in',
 'an',
 'amazing',
 'country',
 '.']

In [8]:
list(zip(result['checked'], result['pos']))

[('He', 'PRP'),
 ('would', 'MD'),
 ('love', 'VB'),
 ('to', 'TO'),
 ('visit', 'VB'),
 ('many', 'JJ'),
 ('beautiful', 'JJ'),
 ('cities', 'NNS'),
 ('wth', 'NN'),
 ('you', 'PRP'),
 ('.', '.'),
 ('He', 'PRP'),
 ('lives', 'VBZ'),
 ('in', 'IN'),
 ('an', 'DT'),
 ('amazing', 'JJ'),
 ('country', 'NN'),
 ('.', '.')]

In [9]:
list(zip(result['checked'], result['pos']))

[('He', 'PRP'),
 ('would', 'MD'),
 ('love', 'VB'),
 ('to', 'TO'),
 ('visit', 'VB'),
 ('many', 'JJ'),
 ('beautiful', 'JJ'),
 ('cities', 'NNS'),
 ('wth', 'NN'),
 ('you', 'PRP'),
 ('.', '.'),
 ('He', 'PRP'),
 ('lives', 'VBZ'),
 ('in', 'IN'),
 ('an', 'DT'),
 ('amazing', 'JJ'),
 ('country', 'NN'),
 ('.', '.')]